In [1]:
import numpy as np
import matplotlib.pyplot as plt
from bqplot import *
from IPython.display import display
from ipywidgets import interactive, fixed, IntSlider, HTML, HTMLMath, FloatSlider,HBox
from scipy.special import binom

%matplotlib inline

In [2]:
%%javascript
document.body.classList.add('theme-dark')

<IPython.core.display.Javascript object>

In [3]:
def binomial_pmf(n,p,k):
    """
    Computes the binomial pmf 
    - n iid bernoulli experiments are performed
    - each with probability of success p
    - k is the number of success 
    """
    if k <=n and k>=0:
        return binom(n,k) * (p ** k) * ((1-p) ** (n-k) )
    return 0


def binomial_cdf(n,p,K):
    return sum([binomial_pmf(n,p,k) for k in range(K+1)])

# Plotting the PMF

In [35]:
n = 10
p = .5
r = np.array([i for i in range(0,n+1)])
probs = np.array([binomial_pmf(n,p,k) for k in r])

n_slider = IntSlider(max=100, min=1, value=n, description="$n$  Value")
p_slider = FloatSlider(max=1.0, min=0.0, value=p,description="$p$  Value")
    
xs = OrdinalScale()
ys = LinearScale()

bar = Bars(x=r, y=probs, scales={'x': xs, 'y': ys}, padding=0.2, type='grouped')
xax = Axis(scale=xs)
yax = Axis(scale=ys, orientation='vertical', tick_format='0.0', grid_lines='solid')

fig = Figure(marks=[bar], axes=[xax, yax], animation_duration=1000)

def binom_callback(change):
    n = n_slider.value
    p = p_slider.value
    r = np.array([i for i in range(0,n+1)])
    probs = np.array([binomial_pmf(n,p,k) for k in r])
    bar.x = r
    bar.y = probs
    
n_slider.observe(binom_callback, 'value')
p_slider.observe(binom_callback, 'value')
display(fig, n_slider, p_slider)

Figure(animation_duration=1000, axes=[Axis(scale=OrdinalScale()), Axis(orientation='vertical', scale=LinearSca…

IntSlider(value=10, description='$n$  Value', min=1)

FloatSlider(value=0.5, description='$p$  Value', max=1.0)

In [50]:
# Pr(X<=cutoff)
cutoff = 5


# Redefine Sliders
# Distribution Parameters, and cutoff
n_slider = IntSlider(max=100, min=1, value=n, description="$n$  Value")
p_slider = FloatSlider(max=1.0, min=0.0, value=p, description="$p$  Value")
cdf_slider =  IntSlider(min=-1, max=n_slider.value, value=cutoff, description="$Pr(X \leq k )$")



# set up axis scaling 
xs = OrdinalScale()#LinearScale()
xs_cdf = LinearScale()

ys = LinearScale()
cdf_ys = LinearScale(min=0,max=1)

# Axis scaling for PDF graph
xax = Axis(scale=xs, label='x', grid_lines='solid')
yax = Axis(scale=ys, orientation='vertical', tick_format='0.2f', label='y', grid_lines='solid')

# Axis scaling for CDF Graph 
cdf_xax = Axis(scale=xs_cdf, label="x)", grid_lines='solid')
cdf_yax = Axis(scale=cdf_ys, orientation='vertical', tick_format='0.2f', label='y', grid_lines='solid')

# X Values to graph for graph 
# TODO: only include value if it has probability larger than 1e-6
x_vals = np.array([i for i in range(0,n_slider.value +1)])

# PDF and CDF 
pdf_vals = np.array([binomial_pmf(n_slider.value,p_slider.value,k) for k in x_vals])
cdf_vals = np.array([binomial_cdf(n_slider.value,p_slider.value,k) for k in x_vals])

bar = Bars(x=x_vals, y=pdf_vals, scales={'x': xs, 'y': ys}, padding=0.2, type='grouped')

# PDF shade
pdf = Lines(x=x_vals, y=pdf_vals, scales={'x': xs, 'y': ys}, colors=['magenta'])
bool_inclusion = x_vals <= cutoff
pdf_shade_x = x_vals[bool_inclusion]
pdf_shade_y = pdf_vals[bool_inclusion]
pdf_shade =  Lines(x=pdf_shade_x, y=pdf_shade_y, scales={'x': xs, 'y': ys}, colors=['magenta'],fill_colors=['magenta'] ,fill='bottom')

cdf = Lines(x=x_vals, y=cdf_vals, scales={'x': xs, 'y': ys}, colors=['yellow'])
scatt = Scatter(x=[cutoff], y=[ binomial_cdf(n_slider.value,p_slider.value, cdf_slider.value) ], scales={'x': xs, 'y': ys}, colors=['blue'])


fig1 = Figure(marks=[bar,pdf, pdf_shade], axes=[xax, yax], animation_duration=1000)
fig2 = Figure(marks=[cdf, scatt], axes=[cdf_xax, cdf_yax], animation_duration=1000)


def dist_callback(change):
    cutoff = cdf_slider.value
    x_vals = np.array([i for i in range(0,n_slider.value +1)])
    cdf_slider.max = n_slider.value
    # PDF and CDF 
    pdf_vals = np.array([binomial_pmf(n_slider.value,p_slider.value,k) for k in x_vals])
    cdf_vals = np.array([binomial_cdf(n_slider.value,p_slider.value,k) for k in x_vals])
    pdf.x, pdf.y = x_vals, pdf_vals
    cdf.x, cdf.y = x_vals, cdf_vals
    bool_inclusion = x_vals <= cutoff
    pdf_shade_x = x_vals[bool_inclusion]
    pdf_shade_y = pdf_vals[bool_inclusion]
    pdf_shade.x = pdf_shade_x
    pdf_shade.y = pdf_shade_y  
    bar.x = x_vals
    bar.y = pdf_vals
    scatt.x = [cutoff]
    scatt.y = [ binomial_cdf(n_slider.value,p_slider.value, cdf_slider.value) ]

def binom_pdf_cdf_callback(change):
    cutoff = cdf_slider.value
    x_vals = np.array([i for i in range(0,n_slider.value +1)])
    # PDF and CDF 
    pdf_vals = np.array([binomial_pmf(n_slider.value,p_slider.value,k) for k in x_vals])
    cdf_vals = np.array([binomial_cdf(n_slider.value,p_slider.value,k) for k in x_vals])
    bool_inclusion = x_vals <= cutoff
    pdf_shade_x = x_vals[bool_inclusion]
    pdf_shade_y = pdf_vals[bool_inclusion]
    pdf_shade.x = pdf_shade_x
    pdf_shade.y = pdf_shade_y
    bar.x = x_vals
    bar.y = pdf_vals
    scatt.x = [cutoff]
    scatt.y = [ binomial_cdf(n_slider.value,p_slider.value, cdf_slider.value) ]

    
n_slider.observe(dist_callback, 'value')
p_slider.observe(dist_callback, 'value')
cdf_slider.observe(binom_pdf_cdf_callback, 'value')
display(HBox((fig1, fig2) ))
display( n_slider, p_slider, cdf_slider)


IntSlider(value=10, description='$n$  Value', min=1)

FloatSlider(value=0.5, description='$p$  Value', max=1.0)

IntSlider(value=5, description='$Pr(X \\leq k )$', max=10, min=-1)

In [39]:
bar.y

array([0.00097656, 0.00976562, 0.04394531, 0.1171875 , 0.20507812,
       0.24609375, 0.20507812, 0.1171875 , 0.04394531, 0.00976562,
       0.00097656])